# Ejercicio de Procesado de Lenguaje Natural
En este ejercicio vamos a utilizar las técnicas aprendidas de NLP para clasificar la tipología de un conjunto de reclamaciones ciudadanas.  
Utilizaremos un modelo TF-IDF básico, y luego probaremos con bigrams y una reducción de dimensionalidad LSA para ver si obtenemos alguna mejora.

### Importación librerías

In [1]:
import pandas as pd
import numpy as np
import spacy
pd.set_option('display.max_colwidth', -1)
#Importa el resto de librerías necesarias

### Carga de los datos
El archivo CSV de datos tiene tres columnas:  
- Observaciones: el texto (incidencia) a clasificar
- Tipología: la clase (etiqueta) de cada incidencia
- Original: característica binaria que no se usa en este problema

In [2]:
df=pd.read_csv('./Data/incidencias.csv', sep=";")

Muestra información del DataFrame y la cuenta de muestras en cada clase. ¿cuántas clases distintas hay? ¿están balanceadas?

In [3]:
#completar
df.head()

,Tipología,Observaciones,Original
0,Vía Pública,hay coches mal aparcados en la calle de los mayos,N
1,Vía Pública,las calles están llenas de mierdas de los perros por todas partes,N
2,Vía Pública,hay un socavón en la avenida de ademuz,N
3,Vía Pública,Rejilla metálica al lado de la carretera frente a casa de Catalina y Pepe en mal estado. Se ha producido una caída.,S
4,Vía Pública,Cada vez que hay una tormenta se llena la calle de tierra y piedras. Considero que deberían buscar una rápida solución. Gracias,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 3 columns):
Tipología        220 non-null object
Observaciones    220 non-null object
Original         174 non-null object
dtypes: object(3)
memory usage: 5.3+ KB


In [5]:
df.Tipología.value_counts()

Mobiliario Urbano                31
Alumbrado                        31
Parques y jardines               29
Vía Pública                      28
Limpieza                         28
Agua                             27
Alcantarillado                   26
Plagas de insectos y roedores    20
Name: Tipología, dtype: int64

Tenemos 220 valores y parece que están más o menos balanceados, en las cuales tenemos 8 clases distintas.

In [32]:
df.Tipología.count()

220

### Limpieza del texto
Programa una función para limpiar el texto en los siguientes términos:  
- Elimina los números (expresión regular `r'\d+'`)  
- Elimina los signos de puntuación
- Convierte todas las palabras a minúscula

In [6]:
#completar
import re, string, spacy
nlp=spacy.load('es_core_news_md')
pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation)))
def clean_text(text):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    eliminamos los tokens que son signos de puntuación, convertimos en
    minúsculas y quitamos signos de puntuación. Para terminar
    volvemos a convertir en cadena de texto"""
    text = re.sub(r'\d+', '', text) #elimina números
    tokens = nlp(text)
    tokens = [tok.lower_ for tok in tokens if not tok.is_punct]
    filtered_tokens = [pattern2.sub('', token) for token in tokens]
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text

Aplica la función de limpieza a la columna 'Observaciones' del DataFrame

In [7]:
#completar
df.Observaciones=df.Observaciones.apply(clean_text)

In [8]:
df=df[df.Observaciones!='']

In [9]:
df.head()

,Tipología,Observaciones,Original
0,Vía Pública,hay coches mal aparcados en la calle de los mayos,N
1,Vía Pública,las calles están llenas de mierdas de los perros por todas partes,N
2,Vía Pública,hay un socavón en la avenida de ademuz,N
3,Vía Pública,rejilla metálica al lado de la carretera frente a casa de catalina y pepe en mal estado se ha producido una caída,S
4,Vía Pública,cada vez que hay una tormenta se llena la calle de tierra y piedras considero que deberían buscar una rápida solución gracias,S


### Funciones auxiliares
Completa estas funciones para calcular la matriz BoW y TF-IDF del Corpus de texto. La función `train_predict_evaluate_model` ya está definida por ti.

In [33]:
#función para extraer el modelo BoW del corpus
from sklearn.feature_extraction.text import CountVectorizer

def bow_extractor(corpus, ngram_range=(1,1), min_df=1, max_df=1.0):
    """Función que ajusta un modelo BoW sobre un corpus de texto
    y devuelve el modelo y la matriz BoW
    El corpus se debe pasar como una lista de strings."""
    #COMPLETAR
    vectorizer = CountVectorizer(ngram_range=ngram_range, min_df=min_df, max_df=max_df)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

#función para extraer el modelo TF-IDF del corpus
from sklearn.feature_extraction.text import TfidfVectorizer  

def tfidf_extractor(corpus, ngram_range=(1,1), min_df=1, max_df=1.0):
    """Función que ajusta un modelo TF-IDF sobre un corpus de texto
    y devuelve el modelo y la matriz TF-IDF
    El corpus se debe pasar como una lista de strings."""   
    #COMPLETAR
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, min_df=min_df, max_df=max_df)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

from sklearn.metrics import classification_report    
def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    '''Función que entrena un clasificador, lo evalúa sobre un conjunto
    de test, y muestra su rendimiento'''
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    # evaluate model prediction performance   
    print(classification_report(test_labels, predictions))
    return predictions 

### División del conjunto de datos
Divide los datos en un conjunto de entrenamiento (`X_train`, `y_train`) y test (`X_test`, `y_test`) en una proporción 70-30:

In [34]:
#Completar
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Observaciones'], 
                                                    df['Tipología'],
                                                    test_size=0.3,
                                                    random_state=0)

## Clasificación
Entrena un clasificador sobre el conjunto de TRAIN y valida en TEST.  
Prueba con las matrices BoW y TF-IDF, y utiliza los siguientes clasificadores de la librería `scikit-learn`:  
```python
modelLR = LogisticRegression()
modelNB = GaussianNB()
modelSVM = SGDClassifier(loss='hinge', max_iter=100)
```
Guarda las predicciones de todos los modelos (luego la usarás para mostrar la matriz de confusión)

In [35]:
# creamos los modelos
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

modelos = [('Logistic Regression', LogisticRegression()),
           ('Naive Bayes', GaussianNB()),
           ('Linear SVM', SGDClassifier(loss='hinge', max_iter=100))]

### Entrenamos clasificadores con modelos TF-IDF

In [13]:
#calcula la matriz TF-IDF sobre conjunto de entrenamiento y test
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(X_train)
tfidf_test_features = tfidf_vectorizer.transform(X_test)


#entrena y valida los clasificadores
for m, clf in modelos:
    print('Modelo {} con características TD-IDF'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features= tfidf_train_features.toarray(),
                                           train_labels=y_train,
                                           test_features=tfidf_test_features.toarray(),
                                           test_labels=y_test)

Modelo Logistic Regression con características TD-IDF
                               precision    recall  f1-score   support

                         Agua       0.80      0.50      0.62         8
               Alcantarillado       1.00      0.50      0.67        10
                    Alumbrado       0.69      0.82      0.75        11
                     Limpieza       1.00      0.21      0.35        14
            Mobiliario Urbano       0.26      0.71      0.38         7
           Parques y jardines       0.38      1.00      0.56         5
Plagas de insectos y roedores       1.00      1.00      1.00         3
                  Vía Pública       0.40      0.25      0.31         8

                     accuracy                           0.55        66
                    macro avg       0.69      0.62      0.58        66
                 weighted avg       0.73      0.55      0.54        66

Modelo Naive Bayes con características TD-IDF
                               precision    r

### Entrenamos clasificadores con modelos BoW

In [14]:
#calcula la matriz BoW sobre conjunto de entrenamiento y test
bow_vectorizer, bow_train_features = bow_extractor(X_train)
bow_test_features = bow_vectorizer.transform(X_test)
#entrena y valida los clasificadores

for m, clf in modelos:
    print('Modelo {} con características BoW'.format(m))
    BoW_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features= bow_train_features.toarray(),
                                           train_labels=y_train,
                                           test_features=bow_test_features.toarray(),
                                           test_labels=y_test)

Modelo Logistic Regression con características BoW
                               precision    recall  f1-score   support

                         Agua       1.00      0.75      0.86         8
               Alcantarillado       0.45      0.50      0.48        10
                    Alumbrado       0.89      0.73      0.80        11
                     Limpieza       0.88      0.50      0.64        14
            Mobiliario Urbano       0.29      0.71      0.42         7
           Parques y jardines       0.44      0.80      0.57         5
Plagas de insectos y roedores       1.00      1.00      1.00         3
                  Vía Pública       0.67      0.25      0.36         8

                     accuracy                           0.61        66
                    macro avg       0.70      0.66      0.64        66
                 weighted avg       0.71      0.61      0.62        66

Modelo Naive Bayes con características BoW
                               precision    recall 

¿Cuál es el modelo que mejor funciona?  
Muestra la matriz de confusión sobre el conjunto de test usando el siguiente código (debes sustituir `prediccion` por el array de predicciones de tu mejor modelo):

En este caso el modelo que mejor funciona es el SVM con TF-IDF obteniendo un 65% de precisión en la predicción.

In [15]:
#Matriz de confusión para TF-IDF
resultados = pd.DataFrame({'Real': y_test, 'Prediccion': tfidf_predictions})
pd.crosstab(resultados['Real'], resultados['Prediccion'],margins=True)


Prediccion,Agua,Alcantarillado,Alumbrado,Limpieza,Mobiliario Urbano,Parques y jardines,Plagas de insectos y roedores,Vía Pública,All
Real,,,,,,,,,
Agua,7,0,0,0,0,0,0,1,8
Alcantarillado,1,7,1,0,0,1,0,0,10
Alumbrado,0,0,9,0,1,0,1,0,11
Limpieza,0,1,0,9,2,0,0,2,14
Mobiliario Urbano,0,1,0,1,4,1,0,0,7
Parques y jardines,0,1,0,1,1,2,0,0,5
Plagas de insectos y roedores,0,0,0,0,0,0,3,0,3
Vía Pública,0,1,2,0,2,1,0,2,8
All,8,11,12,11,10,5,4,5,66


In [16]:
#Matriz de confusión para BoW
resultados = pd.DataFrame({'Real': y_test, 'Prediccion': BoW_predictions})
pd.crosstab(resultados['Real'], resultados['Prediccion'],margins=True)

Prediccion,Agua,Alcantarillado,Alumbrado,Limpieza,Mobiliario Urbano,Parques y jardines,Plagas de insectos y roedores,Vía Pública,All
Real,,,,,,,,,
Agua,6,0,0,0,0,1,1,0,8
Alcantarillado,1,6,0,0,1,1,0,1,10
Alumbrado,0,0,8,0,0,2,1,0,11
Limpieza,0,1,0,7,5,0,0,1,14
Mobiliario Urbano,0,0,0,1,4,1,0,1,7
Parques y jardines,0,1,0,1,1,2,0,0,5
Plagas de insectos y roedores,0,0,0,0,0,0,2,1,3
Vía Pública,0,2,0,0,2,2,0,2,8
All,7,10,8,9,13,9,4,6,66


Vemos que palabraas son más relevantes en el modelo que hemos escogido

In [50]:
# # obtenemos los nombres de las características numpy array
# feature_names = np.array(tfidf_vectorizer.get_feature_names())

# # Ordenamos los coeficientes del modelo
# sorted_coef_index = modelos(1).coef_[0].argsort()

# # Listamos los 10 coeficientes menores y mayores
# print('Menores Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
# print('Mayores Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))


### Mejoras del clasificador
Como actividad opcional, intenta mejorar los resultados de estos clasificadores utilizando otras técnicas, como por ejemplo:  
- Distintos clasificadores a los utilizados
- Realizando una reducción de dimensionalidad LDA o LSA del texto
- Utilizando una matriz de características basada en Word Vectors o Paragraph Vectors

#### Lematizar

In [51]:
def lemmatize_text(text):
    """Convertimos el texto a tokens, extraemos el lema de cada token
    y volvemos a convertir en cadena de texto"""
    tokens = nlp(text)
    lemmatized_tokens = [tok.lemma_ for tok in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    
    return lemmatized_text

In [19]:
df["lemas"]=df.Observaciones.apply(lemmatize_text)

In [20]:
df.head()

,Tipología,Observaciones,Original,lemas
0,Vía Pública,hay coches mal aparcados en la calle de los mayos,N,haber coche mal aparcar en lo callar de lo mayo
1,Vía Pública,las calles están llenas de mierdas de los perros por todas partes,N,los calle estar lleno de mierda de lo perro por todo partir
2,Vía Pública,hay un socavón en la avenida de ademuz,N,haber uno socavón en lo avenir de ademuz
3,Vía Pública,rejilla metálica al lado de la carretera frente a casa de catalina y pepe en mal estado se ha producido una caída,S,rejilla metálico al lado de lo carretero frente a casar de catalina y pepe en mal estar se haber producir uno caer
4,Vía Pública,cada vez que hay una tormenta se llena la calle de tierra y piedras considero que deberían buscar una rápida solución gracias,S,cada vez que haber uno tormenta se lleno lo callar de tierra y piedra considerar que deber buscar uno rápido solución gracia


In [21]:
X_train_lem, X_test_lem, y_train, y_test = train_test_split(df['lemas'], 
                                                    df['Tipología'],
                                                    test_size=0.3,
                                                    random_state=0)

In [22]:
#calcula la matriz TF-IDF sobre conjunto de entrenamiento y test
tfidf_vectorizer_lem, tfidf_train_features_lem = tfidf_extractor(X_train_lem)
tfidf_test_features_lem = tfidf_vectorizer_lem.transform(X_test_lem)

for m, clf in modelos:
    print('Modelo {} con características BoW'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features= tfidf_train_features_lem.toarray(),
                                           train_labels=y_train,
                                           test_features=tfidf_test_features_lem.toarray(),
                                           test_labels=y_test)


Modelo Logistic Regression con características BoW
                               precision    recall  f1-score   support

                         Agua       1.00      0.62      0.77         8
               Alcantarillado       0.75      0.60      0.67        10
                    Alumbrado       0.83      0.91      0.87        11
                     Limpieza       1.00      0.21      0.35        14
            Mobiliario Urbano       0.40      0.86      0.55         7
           Parques y jardines       0.36      1.00      0.53         5
Plagas de insectos y roedores       0.75      1.00      0.86         3
                  Vía Pública       0.40      0.25      0.31         8

                     accuracy                           0.61        66
                    macro avg       0.69      0.68      0.61        66
                 weighted avg       0.74      0.61      0.59        66

Modelo Naive Bayes con características BoW
                               precision    recall 

Observamos que no existe una mejora en el accuracy lematizando el texto, seguimos teniendo un accuracy máximo del 65%

#### Modelo con Latent Semantic Analysis
Probamos ahora con el algoritmo LSA a ver si funciona mejor

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

def lsa_extractor(corpus, n_dim=50):
    '''(vectorizer, features) = lsa_extractor(corpus, n_dim=50)
    Función que genera un modelo Latent Semantic Analysis
    sobre un corpus de texto (pasado como lista de textos)
    utilizando n_dim dimensiones
    Devuelve el modelo LSA ya entrenado y el vector sobre el corpus'''
    
    tfidf = TfidfVectorizer(use_idf=True)
    svd = TruncatedSVD(n_dim)
    vectorizer = make_pipeline(tfidf, svd, Normalizer(copy=False))
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

Probamos nuestros modelos primero con 50 y con 200 a ver cómo funciona mejor

In [61]:
#Creamos los vectores de características LSA con 50 dim
lsa_vectorizer, lsa_train_features = lsa_extractor(X_train, n_dim=100)  
lsa_test_features = lsa_vectorizer.transform(X_test)
#entrenamos y validamos
for m, clf in modelos:
    print('Modelo {} con características LSA con 100 dim'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features=lsa_train_features,
                                           train_labels=y_train,
                                           test_features=lsa_test_features,
                                           test_labels=y_test)

Modelo Logistic Regression con características LSA con 100 dim
                               precision    recall  f1-score   support

                         Agua       0.78      0.88      0.82         8
               Alcantarillado       0.70      0.70      0.70        10
                    Alumbrado       0.90      0.82      0.86        11
                     Limpieza       1.00      0.43      0.60        14
            Mobiliario Urbano       0.38      0.71      0.50         7
           Parques y jardines       0.44      0.80      0.57         5
Plagas de insectos y roedores       0.60      1.00      0.75         3
                  Vía Pública       0.50      0.25      0.33         8

                     accuracy                           0.65        66
                    macro avg       0.66      0.70      0.64        66
                 weighted avg       0.72      0.65      0.65        66

Modelo Naive Bayes con características LSA con 100 dim
                           

In [63]:
#Creamos los vectores de características LSA con 200 dim
lsa_vectorizer, lsa_train_features = lsa_extractor(X_train, n_dim=200)  
lsa_test_features = lsa_vectorizer.transform(X_test)
#entrenamos y validamos
for m, clf in modelos:
    print('Modelo {} con características LSA con 200 dim'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features=lsa_train_features,
                                           train_labels=y_train,
                                           test_features=lsa_test_features,
                                           test_labels=y_test)

Modelo Logistic Regression con características LSA con 200 dim
                               precision    recall  f1-score   support

                         Agua       0.75      0.75      0.75         8
               Alcantarillado       0.88      0.70      0.78        10
                    Alumbrado       0.82      0.82      0.82        11
                     Limpieza       1.00      0.29      0.44        14
            Mobiliario Urbano       0.33      0.71      0.45         7
           Parques y jardines       0.38      1.00      0.56         5
Plagas de insectos y roedores       1.00      1.00      1.00         3
                  Vía Pública       0.50      0.25      0.33         8

                     accuracy                           0.62        66
                    macro avg       0.71      0.69      0.64        66
                 weighted avg       0.74      0.62      0.62        66

Modelo Naive Bayes con características LSA con 200 dim
                           

C:\Users\Pilarica\Anaconda3\envs\env_prueba2\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seguimos sin mejorar nuestro rendimiento.

#### Modelos n-gram

In [64]:
# Entrenamos el modelo CountVectorizer con el conjunto de entrenamiento
# como conjunto de unigrams y bigrams sobre todos los términos (min_df=1)
vect_ngrams = CountVectorizer(min_df=1, ngram_range=(1,2)).fit(X_train)

X_train_vectorized_Bow_ngram = vect_ngrams.transform(X_train)
X_test_vectorized_Bow_ngram = vect_ngrams.transform(X_test)

len(vect_ngrams.get_feature_names())

2128

Ahora el número de términos se incrementa mucho debido a la gran cantidad de bigramas a considerar. Vemos por ejemplo algunos de los términos considerados:

In [65]:
feature_names = np.array(vect_ngrams.get_feature_names())
np.random.seed(1234)
np.random.choice(feature_names, 10)

array(['esta en', 'muy', 'en fiestsa', 'valencia', 'carretera está',
       'merendero hay', 'adecuada en', 'luces', 'oxidado',
       'tapa alcantarillado'], dtype='<U25')

Vemos si mejoran los clasificadores con esta matriz de características

In [67]:
for m, clf in modelos:
    print('Modelo {} con características BoW unigrams y bigrams'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features=X_train_vectorized_Bow_ngram.toarray(),
                                           train_labels=y_train,
                                           test_features=X_test_vectorized_Bow_ngram.toarray(),
                                           test_labels=y_test)

Modelo Logistic Regression con características BoW unigrams y bigrams
                               precision    recall  f1-score   support

                         Agua       1.00      0.50      0.67         8
               Alcantarillado       0.50      0.50      0.50        10
                    Alumbrado       0.82      0.82      0.82        11
                     Limpieza       0.78      0.50      0.61        14
            Mobiliario Urbano       0.35      0.86      0.50         7
           Parques y jardines       0.57      0.80      0.67         5
Plagas de insectos y roedores       1.00      1.00      1.00         3
                  Vía Pública       0.60      0.38      0.46         8

                     accuracy                           0.62        66
                    macro avg       0.70      0.67      0.65        66
                 weighted avg       0.70      0.62      0.63        66

Modelo Naive Bayes con características BoW unigrams y bigrams
             

En este caso observamos que mejora el modelo un 3% subiendo al 68% utilizando: Modelo Naive Bayes con características BoW unigrams y bigrams

In [68]:
# Entrenamos el modelo CountVectorizer con el conjunto de entrenamiento
# como conjunto de unigrams y bigrams sobre todos los términos (min_df=1)

vect_ngrams = TfidfVectorizer(min_df=1, ngram_range=(1,2)).fit(X_train)

X_train_vectorized_TIIDF_ngrams = vect_ngrams.transform(X_train)
X_test_vectorized_TIIDF_ngrams = vect_ngrams.transform(X_test)

len(vect_ngrams.get_feature_names())

2128

Probamos si existe alguna mejora haciendo un TF-IDF con ngrams

In [69]:
for m, clf in modelos:
    print('Modelo {} con características BoW unigrams y bigrams'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features=X_train_vectorized_TIIDF_ngrams.toarray(),
                                           train_labels=y_train,
                                           test_features=X_test_vectorized_TIIDF_ngrams.toarray(),
                                           test_labels=y_test)

Modelo Logistic Regression con características BoW unigrams y bigrams
                               precision    recall  f1-score   support

                         Agua       0.80      0.50      0.62         8
               Alcantarillado       1.00      0.50      0.67        10
                    Alumbrado       0.60      0.82      0.69        11
                     Limpieza       1.00      0.21      0.35        14
            Mobiliario Urbano       0.32      0.86      0.46         7
           Parques y jardines       0.38      1.00      0.56         5
Plagas de insectos y roedores       1.00      0.67      0.80         3
                  Vía Pública       0.50      0.25      0.33         8

                     accuracy                           0.55        66
                    macro avg       0.70      0.60      0.56        66
                 weighted avg       0.73      0.55      0.53        66

Modelo Naive Bayes con características BoW unigrams y bigrams
             

Vemos que no se produce ninguna mejora

Por lo tanto, el mejor resultado obtenido después de probar distintas soluciones sería un 68% de accuracy obtenido utilizando un Modelo Naive Bayes con características BoW unigrams y bigrams